``` sql
SELECT count(id) FROM `user_login_history` where date(created) >= '2020-09-01'  and date(created) <= '2020-09-30' and is_mobile = 1;
```

In [ ]:

# import necessary library
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [ ]:
user_login_history_table_columns_dict = {
    '1': 'id',
    '2': 'is_mobile',
    '3': 'client_ip',
    '4': 'device_name',
    '5': 'browser_name',
    '6': 'employee_record_id',
    '7': 'ministry_id',
    '8': 'layer_id',
    '9': 'origin_id',
    '10': 'ofice_id',
    '11': 'office_unit_id',
    '12': 'office_unit_organogram',
    '13': 'office_name',
    '14': 'unit_name',
    '15': 'organogram_name',
    '16': 'user_id',
    '17': 'employee_name',
    '18': 'mobile_number',
    '19': 'employee_email',
    '20': 'login_time',
    '21': 'logout_time',
    '22': 'network_information',
    '23': 'created',
    '24': 'modified',
    '25': 'created_by',
    '26': 'modified_by',
    '27': 'token',
    '28': 'device_type',
    '29': 'device_id',
}
user_login_history_table_columns_list = list(user_login_history_table_columns_dict.values())
user_login_history_table_columns = user_login_history_table_columns_list
# for key in nisponno_records_table_columns_dict.keys():
#     value = 

len(user_login_history_table_columns)

### Check column length consistency

In [ ]:
user_login_history_df = pd.read_csv('../Data/user_login_history_24_11_2021.csv')
if len(user_login_history_table_columns) != user_login_history_df.shape[1]: 
    print(f"previous column length: {len(user_login_history_table_columns)}, current column_length {user_login_history_df.shape[1]}")
    raise ValueError('office table attribute added or removed')

In [ ]:
def load_user_login_history_df():
    user_login_history_df = pd.read_csv('../Data/user_login_history_24_11_2021.csv', names=user_login_history_table_columns)
    return user_login_history_df

In [ ]:
user_login_history_df = None
user_login_history_df = load_user_login_history_df()
user_login_history_df.head()

In [ ]:
# observing dataframe size
user_login_history_df.shape
user_login_history_df.size

In [ ]:

# observing primary key
"min_id", min(user_login_history_df.id.values)
"max_id", max(user_login_history_df.id.values)
"table shape", user_login_history_df.shape

In [ ]:
# observing missing values, datatype, dataframe memory size
user_login_history_df.info()


In [ ]:
# observing null value of every attribute
user_login_history_df.isnull().sum()
# nisponno_records_df.upokarvogi.unique()

In [ ]:
# SELECT count(id) FROM `user_login_history` where date(created) >= '2020-09-01'  and date(created) <= '2020-09-30' and is_mobile = 1;

# Extract mandatory attributes for offices graph analysis
user_login_history_df = None
user_login_history_df = load_user_login_history_df()
print(f"Initial dataframe shape {user_login_history_df.shape}")
user_login_history_df = user_login_history_df[['id', 'is_mobile', 'created']]
user_login_history_df = user_login_history_df.loc[user_login_history_df.is_mobile == 1]
# remove null values
user_login_history_df = user_login_history_df.loc[user_login_history_df.created.notnull()]
print(f"dataframe shape after removing operation_date null value {user_login_history_df.shape}")
# add new column: cretead_new as datetime field from operation_date column
user_login_history_df['created'] = pd.to_datetime(user_login_history_df['created'], errors='coerce')
# errors{‘ignore’, ‘raise’, ‘coerce’}, default ‘raise’
    # If ‘raise’, then invalid parsing will raise an exception.
    # If ‘coerce’, then invalid parsing will be set as NaT.
    # If ‘ignore’, then invalid parsing will return the input.
user_login_history_df.shape
# again remove null values based on created field
user_login_history_df = user_login_history_df.loc[user_login_history_df.created.notnull()]
user_login_history_df.shape

In [ ]:
# Extract years and months from created column
created_datetime_index = pd.DatetimeIndex(user_login_history_df['created'])
years = created_datetime_index.year.values.astype(str)
months = created_datetime_index.month.values.astype(str)
type(years)
type(months)
set(years)
set(months)

In [ ]:
# type(years[0])
user_login_history_df['year'] = years
user_login_history_df['month'] = months
user_login_history_df.shape

In [ ]:
# save the new dataframe as csv
user_login_history_df.to_csv('../Cleaned_data/mobile_app_users.csv', index=False)

In [ ]:
# observe number of offices in every year 
user_login_history_year_by = user_login_history_df.groupby('year')['year'].count()
user_login_history_year_by

In [ ]:
plt.hist(user_login_history_df['year'])


plt.xlabel('years')
plt.ylabel('')
plt.title('')
# plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
# plt.xlim(40, 160)
# plt.ylim(0, 0.03)
plt.grid(True)
plt.show()

In [ ]:
month_map ={
    '1': 'January',
    '01': 'January',
    '2': 'February',
    '02': 'February',
    '3': 'March',
    '03': 'March',
    '4': 'April',
    '04': 'April',
    '5': 'May',
    '05': 'May',
    '6': 'June',
    '06': 'June',
    '7': 'July',
    '07': 'July',
    '8': 'August',
    '08': 'August',
    '9': 'September',
    '09': 'September',
    '10': 'October',
    '11': 'November',
    '12': 'December',
    'unknown': 'unknown',
}

In [ ]:
# Generate general_series and drilldown series for bar graph
# general series: year: number of officess
user_login_histroy_year_by = user_login_history_df.groupby('year')

def generate_general_series_drilldown_series(dataframe_year_by, general_series_name):
    general_series = [
        
        {
            'name': general_series_name,
            'colorByPoint': True,
            'data': [],
        }
    ]
    drilldown_series = []

    for year, year_frame in dataframe_year_by:
        year = str(year)
        # year, year_frame.shape
        temporary_dict_general = {'name': year, 'y': year_frame.shape[0], 'drilldown': year}
        general_series[0]['data'].append(temporary_dict_general)
        temporary_dict_drilldown = {
            'name': year,
            'id': year,
            'data': [],
        }
        month_group_by = year_frame.groupby('month')
        for month, month_frame in month_group_by:
          
            # mg, mf.shape[0]
            month = str(month)
            month = month_map[month]

            lst = [month, month_frame.shape[0]]
            # lst = [month, month_frame['upokarvogi'].sum()]
            temporary_dict_drilldown['data'].append(lst)
        drilldown_series.append(temporary_dict_drilldown)
    return general_series, drilldown_series

general_series, drilldown_series = generate_general_series_drilldown_series(user_login_histroy_year_by, 'upokarvogi')


In [ ]:
general_series
drilldown_series

In [ ]:

month_list =['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August','September', 'October','November', 'December', 'unknown' ]
mobile_app_users = {}

dataframe_year_by = user_login_history_df.groupby('year')

for year, year_frame in dataframe_year_by:
    year = str(year)
    # year, year_frame.shape
    
    month_dict = {}

    month_group_by = year_frame.groupby('month')
    for month, month_frame in month_group_by:

        # mg, mf.shape[0]
        month = str(month)
        month = month_map[month]
        month_dict[month] = int(month_frame.shape[0])
    month_dict = dict(sorted(month_dict.items(), key=lambda x:month_list.index(x[0])))
    dic = {'count': int(year_frame.shape[0]), 'month_map': month_dict}
    mobile_app_users[year] = dic

print(mobile_app_users)

In [ ]:
import json

dictionary = mobile_app_users

with open('../Cleaned_data/mobile_app_users.json', 'w', encoding='utf-8') as f:
    json.dump(dictionary, f, ensure_ascii=False, indent=4)

with open('../Cleaned_data/mobile_app_users.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)
    print(data)

# End End End